# 성현 pkl 데이터의 summary를 vector로

- 파일 위치 알아서 수정해야함

In [1]:
import os
import re
import nltk
from nltk import tokenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from transformers import BertModel, BertTokenizer

### 데이터 불러오기

In [45]:
df = pd.read_pickle('/opt/ml/EDA/last_context_df4_last.pkl')
df

,user_id,isbn,rating,age,location_country,book_title,book_author,year_of_publication,publisher,img_url,language,summary,img_path,category
0,0,0,4,36.307395,canada,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg,actresses
1,1,0,7,30.000000,canada,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg,actresses
2,2,0,8,36.307395,canada,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg,actresses
3,3,0,8,36.307395,canada,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg,actresses
4,4,0,9,36.307395,canada,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg,actresses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383489,7728,149565,0,39.000000,usa-california,on becoming childwise,gezzo,1999.0,multnomah,http://images.amazon.com/images/P/1576734218.0...,en,on becoming childwise responds to this need by...,images/1576734218.01.THUMBZZZ.jpg,familyrelationships
383490,47785,149566,0,37.000000,austria,michael k hlmeiers sagen des klassischen alter...,mkã¶hlmeier,1996.0,piper,http://images.amazon.com/images/P/3492223710.0...,de,respektlos macht der autor mit der griechische...,images/3492223710.01.THUMBZZZ.jpg,mythology
383491,4209,149567,0,33.569767,usa-utah,pool hopping,afleming,2000.0,polestarbookpublishers,http://images.amazon.com/images/P/1896095186.0...,en,the fascinating characters in this short story...,images/1896095186.01.THUMBZZZ.jpg,fiction
383492,40779,149568,0,48.000000,spain,la muerte del decano,gtballester,2002.0,planetapublishingcorporation,http://images.amazon.com/images/P/8408044079.0...,en,none,images/8408044079.01.THUMBZZZ.jpg,fiction


In [46]:
column = ['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher', 'img_url', 'language', 'category', 'summary', 'img_path']

In [49]:
df2vec = df[column]

In [51]:
print(df2vec.shape)
df2vec.head()

(383494, 10)


,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg
1,0,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg
2,0,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg
3,0,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg
4,0,clara callan,rbwright,2001.0,harperflamingocanada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,in a small town in canada clara callan relucta...,images/0002005018.01.THUMBZZZ.jpg


In [55]:
df2vec['isbn'].value_counts()

isbn
194       707
20        581
985       487
675       383
1310      320
         ... 
16933       1
16936       1
81163       1
81162       1
149569      1
Name: count, Length: 149570, dtype: int64

In [56]:
df2vec['isbn'].unique()

array([     0,      1,      2, ..., 149567, 149568, 149569])

### 수작업 벡터화 (`text_data.py`의 함수 활용)

In [63]:
def text_to_vector(text, tokenizer, model, device):
    """
    Parameters
    ----------
    text : np.ndarray
        summary_merge로 부터 병합된 요약 자료 데이터를 입력합니다.
    tokenizer : Tokenizer
        텍스트 데이터를 벡터화 하기 위한 토크나이저를 입력합니다.
    model : Pre-trained Language Model
        텍스트 데이터를 벡터화 하기 위한 사전 학습된 언어 모델을 입력합니다.
    device : str
        학습에 사용할 Device를 입력합니다.
    ----------
    """

    global sentence_embedding

    for sent in tokenize.sent_tokenize(text):
        text_ = "[CLS] " + sent + " [SEP]"
        tokenized = tokenizer.tokenize(text_)
        indexed = tokenizer.convert_tokens_to_ids(tokenized)
        segments_idx = [1] * len(tokenized)
        token_tensor = torch.tensor([indexed])
        sgments_tensor = torch.tensor([segments_idx])
        with torch.no_grad():
            outputs = model(token_tensor.to(device), sgments_tensor.to(device))
            encode_layers = outputs[0]
            sentence_embedding = torch.mean(encode_layers[0], dim=0)

    return sentence_embedding.cpu().detach().numpy()

In [64]:
books_text_df = df2vec[['isbn', 'summary']].copy()
books_text_df= books_text_df.drop_duplicates().reset_index(drop=True)
books_text_df['summary'].fillna('None', inplace=True)

Create Item Summary Vector


In [65]:
books_text_df

,isbn,summary
0,0,in a small town in canada clara callan relucta...
1,1,here for the first time in paperback is an out...
2,2,describes the great flu epidemic of 1918 an ou...
3,3,a chinese immigrant who is convinced she is dy...
4,4,essays by respected military historians includ...
...,...,...
149565,149565,on becoming childwise responds to this need by...
149566,149566,respektlos macht der autor mit der griechische...
149567,149567,the fascinating characters in this short story...
149568,149568,none


In [66]:
device = 'cuda'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [67]:
item_summary_vector_list = []

for summary in tqdm(books_text_df['summary']):
    vector = text_to_vector(summary, tokenizer, model, device)
    item_summary_vector_list.append(vector)

100%|██████████| 149570/149570 [25:26<00:00, 97.97it/s] 


In [68]:
books_text_df['item_summary_vector'] = item_summary_vector_list

In [69]:
books_text_df

,isbn,summary,item_summary_vector
0,0,in a small town in canada clara callan relucta...,"[-0.21812889, -0.20391995, 0.06439797, -0.1359..."
1,1,here for the first time in paperback is an out...,"[-0.5386576, 0.12224604, 0.0784375, -0.0562747..."
2,2,describes the great flu epidemic of 1918 an ou...,"[-0.05029923, 0.13056092, -0.17901798, -0.1529..."
3,3,a chinese immigrant who is convinced she is dy...,"[0.121072516, 0.26905596, 0.2554203, -0.283849..."
4,4,essays by respected military historians includ...,"[-0.30873182, 0.0073390007, -0.2382981, -0.293..."
...,...,...,...
149565,149565,on becoming childwise responds to this need by...,"[-0.15238018, 0.53894126, -0.1035657, -0.10182..."
149566,149566,respektlos macht der autor mit der griechische...,"[-0.6432012, 0.066126645, 0.11430664, -0.24458..."
149567,149567,the fascinating characters in this short story...,"[0.11831149, 0.26141438, 0.47210327, -0.080064..."
149568,149568,none,"[0.06324439, 0.08317136, -0.20397471, 0.017179..."


In [73]:
len(books_text_df.iloc[145555,2])

768

In [75]:
books_text_df.to_pickle('/opt/ml/EDA+PP/summary2vec.pkl')

In [76]:
books_text_df['isbn'].values

array([     0,      1,      2, ..., 149567, 149568, 149569])

In [77]:
books_text_df['isbn'].values.reshape(1, -1)

array([[     0,      1,      2, ..., 149567, 149568, 149569]])

In [78]:
vector = np.concatenate([
                        books_text_df['isbn'].values.reshape(1, -1),
                        books_text_df['item_summary_vector'].values.reshape(1, -1)
                        ])

vector 꼴


```
Output exceeds the size limit. Open the full output data in a text editor
array([[0, 1, 2, ..., 149567, 149568, 149569],
       [array([-2.18128890e-01, -2.03919947e-01,  6.43979684e-02, -1.35915935e-01,
                4.01943266e-01, -2.07400113e-01,  1.33493632e-01,  4.96284813e-01,
               -3.48474115e-01, -1.32244363e-01,  2.11999983e-01, -5.33152819e-01,
                2.35748053e-01,  4.32718039e-01, -4.74705398e-01,  3.83882046e-01,

...
                1.10035114e-01, -3.26449931e-01, -1.37789752e-02, -9.98550132e-02,
               -1.59025133e-01, -2.15792954e-01, -1.69954017e-01, -2.88780332e-01,
               -8.13431516e-02, -2.45168107e-03,  3.48413527e-01, -3.13824624e-01],
              dtype=float32)                                                       ]],
      dtype=object)
```